Created an OpenAi Assistant. Using OpenAi function calling mechanism and threads without streaming, appropriate function/runner is invoked in response to user's query.

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.8/326.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00


Creating the assistant

In [ ]:
from openai import OpenAI

# Initialize the OpenAI client
client = OpenAI(api_key=)

# Define the assistant with functions for stock data retrieval and getting transcripts
assistant = client.beta.assistants.create(
    name="stock-bot",
    instructions="You are a stock bot. Use the provided functions to answer questions.",
    model="gpt-3.5-turbo",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_numerical_table",
                "description": "Get the numerical analysis table for a specific stock and year",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "stock_name": {
                            "type": "string",
                            "description": "The name of the stock, e.g., Apple"
                        },
                        "year": {
                            "type": "integer",
                            "description": "The financial year, e.g., 2023"
                        }
                    },
                    "required": ["stock_name", "year"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_transcripts",
                "description": "Get the transcripts for a specific stock",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "stock_name": {
                            "type": "string",
                            "description": "The name of the stock, e.g., Apple"
                        }
                    },
                    "required": ["stock_name"]
                }
            }
        }
    ]
)

print(assistant)


Assistant(id='asst_I9CxFRhMTrkdblabE9RRhrBI', created_at=1718883571, description=None, instructions='You are a stock bot. Use the provided functions to answer questions.', metadata={}, model='gpt-3.5-turbo', name='stock-bot', object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='get_numerical_table', description='Get the numerical analysis table for a specific stock and year', parameters={'type': 'object', 'properties': {'stock_name': {'type': 'string', 'description': 'The name of the stock, e.g., Apple'}, 'year': {'type': 'integer', 'description': 'The financial year, e.g., 2023'}}, 'required': ['stock_name', 'year']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_transcripts', description='Get the transcripts for a specific stock', parameters={'type': 'object', 'properties': {'stock_name': {'type': 'string', 'description': 'The name of the stock, e.g., Apple'}}, 'required': ['stock_name']}), type='function')], response_format='auto', temper

Running the threads without Streamimg

In [ ]:
#function for creating thread,run and message
def create_message_and_run(assistant,query,thread=None):
  if not thread:
    thread = client.beta.threads.create()

  message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=query
  )
  run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
  )
  print(thread)
  print(message)
  print(run)
  return run,thread


In [ ]:
#submit the response of the called function to assistant
def submit_tool_outputs(run,thread,function_id,function_response):
    run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
      {
        "tool_call_id": function_id,
        "output": str(function_response),
      }
    ]

    )
    print(run)

    return run

#provide user_query
query = "give me financial analysis for apple for 2021 "
run,thread = create_message_and_run(assistant=assistant,query=query)

Thread(id='thread_ALfVqAgLhCGcXpRczm3OHbP9', created_at=1718883669, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))
Message(id='msg_Ave5y2nvK9JBICFbu468ywMn', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='give me financial analysis for apple for 2021 '), type='text')], created_at=1718883669, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_ALfVqAgLhCGcXpRczm3OHbP9')
Run(id='run_gFcMmEUwbnFeOrr6oTRBVutt', assistant_id='asst_I9CxFRhMTrkdblabE9RRhrBI', cancelled_at=None, completed_at=None, created_at=1718883669, expires_at=1718884269, failed_at=None, incomplete_details=None, instructions='You are a stock bot. Use the provided functions to answer questions.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo', object='thread.run',

In [ ]:
run

Run(id='run_gFcMmEUwbnFeOrr6oTRBVutt', assistant_id='asst_I9CxFRhMTrkdblabE9RRhrBI', cancelled_at=None, completed_at=None, created_at=1718883669, expires_at=1718884269, failed_at=None, incomplete_details=None, instructions='You are a stock bot. Use the provided functions to answer questions.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_ALfVqAgLhCGcXpRczm3OHbP9', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_numerical_table', description='Get the numerical analysis table for a specific stock and year', parameters={'type': 'object', 'properties': {'stock_name': {'type': 'string', 'description': 'The name of the stock, e.g., Apple'}, 'year': {'type': 'integer', 'description': 'The financial year, e.g., 2023'}}, 'required': ['stock_name', 'year']}), type=

In [ ]:
thread

Thread(id='thread_ALfVqAgLhCGcXpRczm3OHbP9', created_at=1718883669, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [ ]:
run.status

'queued'

get details of the appropriate function called

In [ ]:

def get_function_details(run):

  print("\nrun.required_action\n",run.required_action)#

  function_name = run.required_action.submit_tool_outputs.tool_calls[0].function.name
  arguments = run.required_action.submit_tool_outputs.tool_calls[0].function.arguments
  function_id = run.required_action.submit_tool_outputs.tool_calls[0].id

  print(f"function_name: {function_name} and arguments: {arguments}")

  return function_name, arguments, function_id

In [ ]:
#function definitions of the functions mentioned in tools
def get_numerical_table(stock_name,year):
  #mongo query
  return 51
def get_transcripts(stock_name):
  print(stock_name)
  pass

invoke the appropriate function based on user_query

In [ ]:

import json
def execute_function_call(function_name,arguments):
    available_functions={"get_numerical_table":get_numerical_table, "get_transcripts":get_transcripts}
    function = available_functions.get(function_name,None)
    if function:
        arguments = json.loads(arguments)
        results = function(**arguments)
        print(arguments)
    else:
        results = f"Error: function {function_name} does not exist"
    return results

Running the thread & submitting the responses back to assistant

In [ ]:
#running the thread & assistant.
import time
while True:

    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    print(run)
    print("run status", run.status)

    if run.status=="requires_action":

        function_name, arguments, function_id  = get_function_details(run)

        function_response = execute_function_call(function_name,arguments)
        print(function_response)
        run = submit_tool_outputs(run,thread,function_id,function_response) # submitting the response back to assistant
        print(run)
        #continue
    if run.status=="completed":

        messages = client.beta.threads.messages.list(thread_id=thread.id)
        print(messages) # print the messages exchanged in the thread
        latest_message = messages.data[0]
        text = latest_message.content[0].text.value
        #print(text)
        break

        # user_input = input()
        # if user_input == "STOP":
        #   break

        # run,thread = create_message_and_run(assistant=assistant,query=user_input,thread=thread)

        # continue;
    time.sleep(5)#needed for async calls


Run(id='run_gFcMmEUwbnFeOrr6oTRBVutt', assistant_id='asst_I9CxFRhMTrkdblabE9RRhrBI', cancelled_at=None, completed_at=None, created_at=1718883669, expires_at=1718884269, failed_at=None, incomplete_details=None, instructions='You are a stock bot. Use the provided functions to answer questions.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-3.5-turbo', object='thread.run', parallel_tool_calls=True, required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_L7D8ExLjrs20WfmaZm2rbh68', function=Function(arguments='{"stock_name":"Apple","year":2021}', name='get_numerical_table'), type='function')]), type='submit_tool_outputs'), response_format='auto', started_at=1718883670, status='requires_action', thread_id='thread_ALfVqAgLhCGcXpRczm3OHbP9', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_numerical_table', description='Get the numerical an